In [89]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import altair as alt

In [90]:
team_id = 251
team_color = "#bf5700"
season = 2023
url = f"http://www.espn.com/womens-college-basketball/team/schedule/_/id/{team_id}/season/{season}"


# Pandas Method

In [91]:
# table = pd.read_html(url)[0]
# table.head()

# BS Method

In [92]:
page = rq.get(url)

In [93]:
soup = bs(page.content)

In [94]:
table = soup.table
data = {'Opponent': [],
        'Attendance': [],
        'Location': []} 
for row in table.find_all('tr')[2:]:
    row_cells = row.find_all('td')
    try:
        opponent = row_cells[1].find_all('a')[1].contents[0]
        game_link = row_cells[2].find('a')['href']
        if(row_cells[1].find_all('span')[2].contents[-1] == "*"):
            location = "Neutral"
        else:
            location = "Home" if row_cells[1].find_all('span')[0].contents[0] == "vs" else "Away"
    except:
        break
    # print(game_link)
    game_page = rq.get(game_link)
    game_soup = bs(game_page.content)
    atndnce = game_soup.find('div', {"class":"Attendance__Numbers"})
    atndnce = atndnce.contents[4] if atndnce != None else 0
    atndnce = ''.join(filter(str.isdigit, atndnce)) if atndnce != 0 else atndnce
    # add current row data to dictionary
    if atndnce != 0:
        data["Opponent"].append(opponent)
        data["Attendance"].append(float(atndnce))
        data["Location"].append(location)

df= pd.DataFrame(data)

In [95]:
df

,Opponent,Attendance,Location
0,Louisiana,5658.0,Home
1,UConn,10167.0,Away
2,Marquette,376.0,Neutral
3,Louisville,365.0,Neutral
4,Rutgers,155.0,Neutral
5,Princeton,5137.0,Home
6,South Florida,5021.0,Home
7,Southern,4987.0,Home
8,Alabama State,5116.0,Home
9,Jackson State,1200.0,Neutral


Altair

In [102]:
alt.Chart(df).mark_bar().encode(
    x=alt.X('Opponent',sort = data["Opponent"]),
    y='Attendance',
    color=alt.Color('Location',
                    scale=alt.Scale(
                        domain=['Home', 'Away' ,'Neutral'],
                        range=[team_color, 'darkgray', 'lightgray']))
)


alt.Chart(...)